# Phishing Email Detector

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# XGBoost
import xgboost as xgb

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# For text tokenization
from collections import Counter
import re

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

DATA_PATH = Path("data/preprocessed/emails_combined.csv")
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

# Feature columns for ML models
FEATURE_COLS = [
    'num_words', 'num_unique_words', 'num_stopwords', 'num_links',
    'num_unique_domains', 'num_email_addresses', 'num_spelling_errors',
    'num_urgent_keywords'
]

# Deep Learning parameters
MAX_VOCAB_SIZE = 10000  # Vocabulary size
MAX_LEN = 200          # Maximum sequence length
EMBEDDING_DIM = 128    # Embedding dimension
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


## Email Class

In [2]:
class EmailDataset(Dataset):
    """PyTorch Dataset for email classification."""
    
    def __init__(self, texts, labels, vocab=None, max_len=MAX_LEN):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        
        # Build or use existing vocabulary
        if vocab is None:
            self.vocab = self.build_vocab(texts)
        else:
            self.vocab = vocab
    
    def build_vocab(self, texts):
        """Build vocabulary from texts."""
        word_counts = Counter()
        for text in texts:
            words = self.tokenize(text)
            word_counts.update(words)
        
        # Get most common words
        most_common = word_counts.most_common(MAX_VOCAB_SIZE - 2)  # Reserve for PAD and UNK
        
        # Build vocab dictionary
        vocab = {'<PAD>': 0, '<UNK>': 1}
        for idx, (word, _) in enumerate(most_common, start=2):
            vocab[word] = idx
        
        return vocab
    
    def tokenize(self, text):
        """Simple tokenization."""
        text = text.lower()
        words = re.findall(r'\b\w+\b', text)
        return words
    
    def text_to_sequence(self, text):
        """Convert text to sequence of indices."""
        words = self.tokenize(text)
        sequence = [self.vocab.get(word, self.vocab['<UNK>']) for word in words]
        
        # Truncate or pad
        if len(sequence) > self.max_len:
            sequence = sequence[:self.max_len]
        else:
            sequence = sequence + [self.vocab['<PAD>']] * (self.max_len - len(sequence))
        
        return sequence
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        
        sequence = self.text_to_sequence(text)
        
        return torch.tensor(sequence, dtype=torch.long), torch.tensor(label, dtype=torch.float32)


## Models

### DL and ML Models

#### DL

In [3]:
class LSTMClassifier(nn.Module):
    """LSTM model for text classification."""
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=2, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, 
                           batch_first=True, dropout=dropout, bidirectional=True)
        self.fc1 = nn.Linear(hidden_dim * 2, 64)  # *2 for bidirectional
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)
        
        # LSTM
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        # Use last hidden state (concatenate forward and backward)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        # Fully connected layers
        out = self.fc1(hidden)
        out = torch.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        
        return out.squeeze()

class CNNClassifier(nn.Module):
    """CNN model for text classification."""
    
    def __init__(self, vocab_size, embedding_dim, num_filters=128, filter_sizes=[3, 4, 5], dropout=0.5):
        super(CNNClassifier, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # Multiple convolutional layers with different filter sizes
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=fs)
            for fs in filter_sizes
        ])
        
        self.fc1 = nn.Linear(len(filter_sizes) * num_filters, 64)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)
        
        # Permute for Conv1d: (batch_size, embedding_dim, seq_len)
        embedded = embedded.permute(0, 2, 1)
        
        # Apply convolution and max pooling
        conved = [torch.relu(conv(embedded)) for conv in self.convs]
        pooled = [torch.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        # Concatenate pooled outputs
        cat = torch.cat(pooled, dim=1)
        
        # Fully connected layers
        out = self.fc1(cat)
        out = torch.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        
        return out.squeeze()
    

def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for sequences, labels in dataloader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    
    return avg_loss, accuracy

def evaluate(model, dataloader, criterion, device):
    """Evaluate model."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for sequences, labels in dataloader:
            sequences, labels = sequences.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            
            # Statistics
            total_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            # Store predictions
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    
    return avg_loss, accuracy, np.array(all_labels), np.array(all_preds), np.array(all_probs)

def train_dl_models(X_text_train, X_text_test, y_train, y_test):
    """Train and evaluate deep learning models using PyTorch: LSTM and CNN."""
    
    # Create datasets
    print("Preparing datasets...")
    train_dataset = EmailDataset(X_text_train.reset_index(drop=True), 
                                 y_train.reset_index(drop=True))
    test_dataset = EmailDataset(X_text_test.reset_index(drop=True), 
                                y_test.reset_index(drop=True),
                                vocab=train_dataset.vocab)
    
    # Create validation split from training data
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        train_dataset, [train_size, val_size]
    )
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"✅ Vocabulary size: {len(test_dataset.vocab)}")
    print(f"✅ Train samples: {len(train_dataset)}")
    print(f"✅ Validation samples: {len(val_dataset)}")
    print(f"✅ Test samples: {len(test_dataset)}")
    
    # Dictionary to store results
    dl_results = {}
    
    # Loss function
    criterion = nn.BCELoss()
    
    # ========================================================================
    # 1. LSTM Model
    # ========================================================================
    print("\n" + "="*70)
    print("1. LSTM MODEL")
    print("="*70)
    
    lstm_model = LSTMClassifier(
        vocab_size=len(test_dataset.vocab),
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=64,
        num_layers=2,
        dropout=0.5
    ).to(device)
    
    print(f"Model architecture:")
    print(lstm_model)
    
    optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE)

    print(f"Training LSTM for {EPOCHS} epochs...")
    best_val_loss = float('inf')
    patience = 3
    patience_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_epoch(lstm_model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, _, _, _ = evaluate(lstm_model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{EPOCHS} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(lstm_model.state_dict(), RESULTS_DIR / 'best_lstm.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # Load best model
    lstm_model.load_state_dict(torch.load(RESULTS_DIR / 'best_lstm.pth'))
    
    print("\n✅ Training complete!")
    
    # Evaluate on test set
    _, _, y_true_lstm, y_pred_lstm, y_prob_lstm = evaluate(lstm_model, test_loader, criterion, device)
    
    dl_results['LSTM'] = evaluate_model(
        y_true_lstm, y_pred_lstm, y_prob_lstm, "LSTM"
    )
    
    # ========================================================================
    # 2. CNN Model
    # ========================================================================
    print("\n" + "="*70)
    print("2. CNN MODEL")
    print("="*70)
    
    cnn_model = CNNClassifier(
        vocab_size=len(test_dataset.vocab),
        embedding_dim=EMBEDDING_DIM,
        num_filters=128,
        filter_sizes=[3, 4, 5],
        dropout=0.5
    ).to(device)

    print(f"Model architecture:")
    print(cnn_model)
    
    optimizer = optim.Adam(cnn_model.parameters(), lr=LEARNING_RATE)

    print(f"Training CNN for {EPOCHS} epochs...")
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_epoch(cnn_model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, _, _, _ = evaluate(cnn_model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{EPOCHS} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(cnn_model.state_dict(), RESULTS_DIR / 'best_cnn.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # Load best model
    cnn_model.load_state_dict(torch.load(RESULTS_DIR / 'best_cnn.pth'))
    
    print("\n✅ Training complete!")
    print("\n📊 Evaluating on test set...")
    
    # Evaluate on test set
    _, _, y_true_cnn, y_pred_cnn, y_prob_cnn = evaluate(cnn_model, test_loader, criterion, device)
    
    dl_results['CNN'] = evaluate_model(
        y_true_cnn, y_pred_cnn, y_prob_cnn, "CNN"
    )
    
    print("\n" + "="*70 + "\n")
    
    return dl_results, (lstm_model, cnn_model)

##### ML

In [4]:
def train_ml_models(X_train, X_test, y_train, y_test):
    """Train and evaluate ML models: Logistic Regression, Random Forest, XGBoost."""
   
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Dictionary to store results
    ml_results = {}
    
    # ========================================================================
    # 1. Logistic Regression
    # ========================================================================
    print("\n" + "="*70)
    print("1. LOGISTIC REGRESSION")
    print("="*70)

    print("Training Logistic Regression...")
    lr_model = LogisticRegression(max_iter=1000, random_state=42)
    lr_model.fit(X_train_scaled, y_train)
    
    print("✅ Training complete!")
    
    # Predictions
    y_pred_lr = lr_model.predict(X_test_scaled)
    y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]
    
    # Evaluate
    ml_results['Logistic Regression'] = evaluate_model(
        y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression"
    )
    
    # ========================================================================
    # 2. Random Forest
    # ========================================================================
    print("\n" + "="*70)
    print("2. RANDOM FOREST")
    print("="*70)
    
    print("Training Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=100, 
        max_depth=20,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    
    print("✅ Training complete!")
    
    # Predictions
    y_pred_rf = rf_model.predict(X_test)
    y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]
    
    # Evaluate
    ml_results['Random Forest'] = evaluate_model(
        y_test, y_pred_rf, y_pred_proba_rf, "Random Forest"
    )
    
    # Feature importance
    print("Feature Importance (Random Forest):")
    feature_importance = pd.DataFrame({
        'feature': FEATURE_COLS,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    print(feature_importance)
    
    # ========================================================================
    # 3. XGBoost
    # ========================================================================
    print("\n" + "="*70)
    print("3. XGBOOST")
    print("="*70)

    print("Training XGBoost...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        eval_metric='logloss'
    )
    xgb_model.fit(X_train, y_train)
    
    print("✅ Training complete!")
    
    # Predictions
    y_pred_xgb = xgb_model.predict(X_test)
    y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
    
    # Evaluate
    ml_results['XGBoost'] = evaluate_model(
        y_test, y_pred_xgb, y_pred_proba_xgb, "XGBoost"
    )
    
    # Feature importance
    print("Feature Importance (XGBoost):")
    feature_importance_xgb = pd.DataFrame({
        'feature': FEATURE_COLS,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    print(feature_importance_xgb)
    
    print("\n" + "="*70 + "\n")
    
    return ml_results, (lr_model, rf_model, xgb_model)

## Training

In [5]:
def load_and_split_data(test_size=0.2, random_state=42):
    """Load the preprocessed data and split into train/test sets."""
    
    # Load data
    print(f"Loading data from: {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)

    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")

    # Check for missing values
    print(f"\nMissing values:")
    print(df.isnull().sum())
    
    # Class distribution
    print(f"\nClass distribution:")
    print(df['label'].value_counts())
    print(f"Class balance:")
    print(df['label'].value_counts(normalize=True))
    
    # Split data
    print(f"\nSplitting data (test_size={test_size})...")
    
    X = df[FEATURE_COLS]  # Features for ML
    X_text = df['text']    # Text for DL
    y = df['label']        # Labels
    
    # Stratified split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    X_text_train, X_text_test, _, _ = train_test_split(
        X_text, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    print(f"✅ Train set: {X_train.shape[0]} samples")
    print(f"✅ Test set: {X_test.shape[0]} samples")
    
    return X_train, X_test, X_text_train, X_text_test, y_train, y_test

### Evaluate Model

In [6]:
def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """Evaluate model performance and print metrics."""
    
    print(f"Evaluation Metrics for {model_name}:")
    print("-" * 70)
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")
    
    # Confusion matrix
    print(f"Confusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    
    # Classification report
    print(f"Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Legitimate', 'Phishing']))
    
    # Store results
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'confusion_matrix': cm,
        'y_true': y_true,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    return results


In [7]:
# Load dataset
X_train, X_test, X_text_train, X_text_test, y_train, y_test = load_and_split_data(test_size=0.2, random_state=42)

Loading data from: data/preprocessed/emails_combined.csv
Dataset shape: (23609, 10)
Columns: ['text', 'label', 'num_words', 'num_unique_words', 'num_stopwords', 'num_links', 'num_unique_domains', 'num_email_addresses', 'num_spelling_errors', 'num_urgent_keywords']

Missing values:
text                   0
label                  0
num_words              0
num_unique_words       0
num_stopwords          0
num_links              0
num_unique_domains     0
num_email_addresses    0
num_spelling_errors    0
num_urgent_keywords    0
dtype: int64

Class distribution:
label
0    16306
1     7303
Name: count, dtype: int64
Class balance:
label
0    0.690669
1    0.309331
Name: proportion, dtype: float64

Splitting data (test_size=0.2)...
✅ Train set: 18887 samples
✅ Test set: 4722 samples


In [8]:

# Train ML models and get their results and the trained model objects
ml_results, (lr_model, rf_model, xgb_model) = train_ml_models(X_train, X_test, y_train, y_test)



1. LOGISTIC REGRESSION
Training Logistic Regression...
✅ Training complete!
Evaluation Metrics for Logistic Regression:
----------------------------------------------------------------------
Accuracy:  0.6999
Precision: 0.6196
Recall:    0.0780
F1-Score:  0.1386
ROC-AUC:   0.6709
Confusion Matrix:
[[3191   70]
 [1347  114]]
Classification Report:
              precision    recall  f1-score   support

  Legitimate       0.70      0.98      0.82      3261
    Phishing       0.62      0.08      0.14      1461

    accuracy                           0.70      4722
   macro avg       0.66      0.53      0.48      4722
weighted avg       0.68      0.70      0.61      4722


2. RANDOM FOREST
Training Random Forest...
✅ Training complete!
Evaluation Metrics for Random Forest:
----------------------------------------------------------------------
Accuracy:  0.7956
Precision: 0.7175
Recall:    0.5599
F1-Score:  0.6290
ROC-AUC:   0.8388
Confusion Matrix:
[[2939  322]
 [ 643  818]]
Classification

In [9]:
# Train DL models and get their results and the trained model objects
dl_results, (lstm_model, cnn_model) = train_dl_models(
    X_text_train, X_text_test, y_train, y_test
)

Preparing datasets...
✅ Vocabulary size: 10000
✅ Train samples: 15109
✅ Validation samples: 3778
✅ Test samples: 4722

1. LSTM MODEL
Model architecture:
LSTMClassifier(
  (embedding): Embedding(10000, 128, padding_idx=0)
  (lstm): LSTM(128, 64, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Training LSTM for 10 epochs...
Epoch 1/10 - Train Loss: 0.3326, Train Acc: 0.8549 | Val Loss: 0.2040, Val Acc: 0.9142
Epoch 2/10 - Train Loss: 0.1447, Train Acc: 0.9480 | Val Loss: 0.1530, Val Acc: 0.9442
Epoch 3/10 - Train Loss: 0.0787, Train Acc: 0.9705 | Val Loss: 0.1274, Val Acc: 0.9510
Epoch 4/10 - Train Loss: 0.0523, Train Acc: 0.9799 | Val Loss: 0.1264, Val Acc: 0.9574
Epoch 5/10 - Train Loss: 0.0359, Train Acc: 0.9860 | Val Loss: 0.1950, Val Acc: 0.9510
Epoch 6/10 - Train Loss: 0.0353, Tra

In [10]:
def compare_all_models(ml_results, dl_results):
    """Compare all models and visualize results."""
    
    # Combine results
    all_results = {**ml_results, **dl_results}
    
    # Create comparison dataframe
    comparison_df = pd.DataFrame({
        'Model': list(all_results.keys()),
        'Accuracy': [r['accuracy'] for r in all_results.values()],
        'Precision': [r['precision'] for r in all_results.values()],
        'Recall': [r['recall'] for r in all_results.values()],
        'F1-Score': [r['f1'] for r in all_results.values()],
        'ROC-AUC': [r['roc_auc'] for r in all_results.values()]
    })
    
    # Add approach column
    comparison_df['Approach'] = comparison_df['Model'].apply(
        lambda x: 'ML' if x in ml_results else 'DL'
    )

    print("Model Comparison:")
    print("="*70)
    print(comparison_df.to_string(index=False))
    
    # Save to CSV
    comparison_df.to_csv(RESULTS_DIR / "model_comparison.csv", index=False)
    print(f"Saved comparison to: {RESULTS_DIR / 'model_comparison.csv'}")
    
    # Visualizations
    print("Creating visualizations...")
    create_visualizations(comparison_df, all_results)
    
    return comparison_df

def create_visualizations(comparison_df, all_results):
    """Create comparison visualizations."""

    # 1. Metrics comparison bar plot
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('ML vs DL Model Comparison', fontsize=16, fontweight='bold')

    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

    for idx, metric in enumerate(metrics):
        ax = axes[idx // 3, idx % 3]
        ax = axes[idx // 3, idx % 3]
        colors = ['skyblue' if a == 'ML' else 'lightcoral' for a in comparison_df['Approach']]
        bars = ax.bar(comparison_df['Model'], comparison_df[metric], color=colors)
        ax.set_title(metric, fontsize=14, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12)
        ax.set_ylim([0, 1.0])
        ax.grid(axis='y', alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=10)
            
    # Legend in the last subplot
    axes[1, 2].axis('off')
    axes[1, 2].legend(
        handles=[
            plt.Rectangle((0,0),1,1, fc='skyblue', label='ML Approach'),
            plt.Rectangle((0,0),1,1, fc='lightcoral', label='DL Approach')
            ],
            loc='center',
            fontsize=12
        )
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'metrics_comparison.png', dpi=300, bbox_inches='tight')
    print(f" ✓ Saved: {RESULTS_DIR / 'metrics_comparison.png'}")
    plt.close()

    # 2. ROC Curves comparison
    fig, ax = plt.subplots(figsize=(10, 8))
    cmap = plt.get_cmap('tab10')
    num_models = len(all_results)
    for idx, (model_name, results) in enumerate(all_results.items()):
        # try/except for cases where roc cannot be computed
        try:
            fpr, tpr, _ = roc_curve(results['y_true'], results['y_pred_proba'])
            auc = results['roc_auc']
            ax.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.3f})', linewidth=2, color=cmap(idx % 10))
        except Exception:
            print(f"Could not plot ROC for {model_name}")
    ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    ax.set_title('ROC Curves - All Models', fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=10)
    plt.savefig(RESULTS_DIR / 'roc_curves_all_models.png', dpi=300, bbox_inches='tight')
    print(f" ✓ Saved: {RESULTS_DIR / 'roc_curves_all_models.png'}")
    plt.close()

In [11]:
# Compare all results and generate final reports and visualizations
comparison_df = compare_all_models(ml_results, dl_results)

Model Comparison:
              Model  Accuracy  Precision   Recall  F1-Score  ROC-AUC Approach
Logistic Regression  0.699915   0.619565 0.078029  0.138602 0.670872       ML
      Random Forest  0.795637   0.717544 0.559890  0.628989 0.838759       ML
            XGBoost  0.767895   0.688728 0.455852  0.548600 0.816012       ML
               LSTM  0.955527   0.903288 0.958932  0.930279 0.990578       DL
                CNN  0.962092   0.910897 0.972621  0.940748 0.994645       DL
Saved comparison to: results/model_comparison.csv
Creating visualizations...
 ✓ Saved: results/metrics_comparison.png
 ✓ Saved: results/roc_curves_all_models.png
